### Imports

In [1]:
import sys 
sys.path.append('C:\\Users\\Eesh Gupta\\Documents\\RU Research\\Chakram')

In [2]:
from BrokenECD import BatchOptimizer as BO_ECD
from DECD_Old import BatchOptimizer as BO_DECD

C:\Users\Eesh Gupta\Documents\RU Research\Chakram\Double-ECD\1Rotation_2ECD\Single ECD Initial Param\BrokenECD.py:145: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if (filename is not None and filename is not "")
C:\Users\Eesh Gupta\Documents\RU Research\Chakram\Double-ECD\1Rotation_2ECD\Single ECD Initial Param\BrokenECD.py:581: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  time_per_epoch = elapsed_time_s / epoch if epoch is not 0 else 0.0



Need tf version 2.3.0 or later. Using tensorflow version: 2.7.0



In [3]:
#%%
# note: timestamp can't use "/" character for h5 saving.
TIMESTAMP_FORMAT = "%Y-%m-%d %H:%M:%S"
END_OPT_STRING = "\n" + "=" * 60 + "\n"
import numpy as np
import tensorflow as tf

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)  # supress warnings
import h5py

# print(
#     "\nNeed tf version 2.3.0 or later. Using tensorflow version: "
#     + tf.__version__
#     + "\n"
# )
import ECD_control.ECD_optimization.tf_quantum as tfq
from ECD_control.ECD_optimization.visualization import VisualizationMixin
import qutip as qt
import datetime
import time

### Single ECD g0 ->g1

Initial Testing Code

In [4]:
#The target oscillator state.
N =10
psi_i = qt.basis(N, 0) # initial state
psi_t = qt.basis(N,1) #target state

#Opt Params
#Optimization of ECD Circuit parameters (betas, phis, and thetas)
#the optimization options
ecd_opt_params = {
'N_blocks' : 5, #circuit depth
'N_multistart' : 10, #Batch size (number of circuit optimizations to run in parallel)
'epochs' : 1000, #number of epochs before termination
'epoch_size' : 10, #number of adam steps per epoch
'learning_rate' : 0.01, #adam learning rate
'term_fid' : 0.9999, #terminal fidelitiy
'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
'beta_scale' : 3.0, #maximum |beta| for random initialization
'initial_states' : [qt.tensor(qt.basis(2,0),psi_i)], #qubit tensor oscillator, start in |g> |0>
'target_states' : [qt.tensor(qt.basis(2,0), psi_t)], #end in |e> |target>.
'name' : 'Fock %d' % 1, #name for printing and saving
'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}


#note: optimizer includes pi pulse in every ECD step. However, final ECD step is implemented 
#in experiment as a displacement since the qubit and oscillator should be disentangled at this point.
#So, we ask the optimizer to end in |e> |target> instead of |g>|target>.

In [5]:
#Optimizer 
tf.config.run_functions_eagerly(True)
ecd_opt = BO_ECD(**ecd_opt_params)
ecd_opt.print_info()

optimization_type: state transfer
N_multistart: 10
N_blocks: 5
term_fid: 0.9999
dfid_stop: 1e-06
no_CD_end: False
learning_rate: 0.01
epoch_size: 10
epochs: 1000
beta_scale: 3.0
alpha_scale: 1.0
theta_scale: 3.141592653589793
use_displacements: False
use_phase: False
name: Fock 1
comment: 
N_cav: 10
filename: Fock 1.h5

Best circuit parameters found:
betas:         [ 0.70222+0.52381j  1.14988-1.31027j -1.19183-0.2395j  -2.38182+0.0419j
  2.29496-0.53759j]
alphas:        [0.+0.j]
phis (deg):    [   0.       -30.94934 -163.14322    9.81318  100.27954]
thetas (deg):  [ 64.59715 170.80888  45.71578  58.84772  12.81453]
Max Fidelity:  0.120082




In [6]:
ecd_opt.optimize()

Start time: 2022-05-20 05:58:29
 Epoch: 77 / 1000 Max Fid: 0.645031 Avg Fid: 0.512413 Max dFid: 0.000026 Avg dFid: 0.000006 Elapsed time: 0:00:19.654946 Remaing time: 0:03:55.604094

KeyboardInterrupt: 

In [ ]:
ecd_opt.best_circuit()

### Feeding ECD params to DECD

#### Convert ECD angles to DECD angles
Add gamma

In [ ]:
#obtain ECD params
circ = ecd_opt.best_circuit()
betas = circ['betas']
alphas = circ['alphas']
phis = circ['phis']
thetas = circ['thetas']

#gammas (displacement of second mode)
gammas = np.copy(betas)
for i in range(len(gammas)):
    gammas[i] = 0.0 +0.0j

#etas (just ignore this)
etas = np.copy(betas)
for i in range(len(etas)):
    etas[i] = np.pi/2 #pi/2

#new initial params (Needs 2 alphas for mode 1 and mode 2)
init_params = [betas, gammas, alphas, alphas, phis, etas, thetas]

In [ ]:
etas

In [ ]:
#Since DECD has more than 1 multistart, we will make sure all multistarts
# have the same initial params-- namely the ecd onces
N_multistart = 10
init_params_more_multi = [np.array([i for j in range(N_multistart)]) for i in init_params]

In [ ]:
init_params_more_multi

#### Initial Testing Code

In [ ]:
#The target oscillator state.
N1 =10
N2 =10
Fock1 = 0
Fock2= 0
psi_i1 = qt.basis(N1,Fock1) #target state
psi_i2 = qt.basis(N2,Fock2)
psi_initial = qt.tensor(psi_i1, psi_i2)

In [ ]:
Fock1 = 1
Fock2= 0
psi_t1 = qt.basis(N1,Fock1) #target state
psi_t2 = qt.basis(N2,Fock2)
psi_target = qt.tensor(psi_t1, psi_t2)
psi_target

In [ ]:
#Optimization of ECD Circuit parameters (betas, phis, and thetas)
#the optimization options
decd_opt_params = {
'N_blocks' : 5, #circuit depth
'N_multistart' : 1, #Batch size (number of circuit optimizations to run in parallel)
'epochs' : 100, #number of epochs before termination
'epoch_size' : 20, #number of adam steps per epoch
'learning_rate' : 0.1, #adam learning rate
'term_fid' : 0.05, #terminal fidelitiy
'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
'beta_scale' : 3.0, #maximum |beta| for random initialization
'gamma_scale' : 3.0, #maximum |gamma| for random initialization
'N_cav1': N1, #number of levels in mode 1
'N_cav2': N2, #number of levels in mode 2
'initial_states' : [qt.tensor(qt.basis(2,0),psi_initial)], #qubit tensor oscillator, start in |g> |0>
'target_states' : [qt.tensor(qt.basis(2,0), psi_target)], #end in |e> |target>.
#"initial_params": init_params,
'name' : 'Fock1 %d' % Fock1, #name for printing and saving
'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}



#### Running DECD

In [ ]:
decd_opt = BO_DECD(**decd_opt_params)
tf.config.run_functions_eagerly(True) # for some reason, important
decd_opt.print_info()

In [ ]:
decd_opt.optimize()

In [ ]:
decd_opt.batch_fidelities(decd_opt.betas_rho,
                decd_opt.betas_angle,
                decd_opt.gammas_rho,
                decd_opt.gammas_angle,
                decd_opt.alphas1_rho,
                decd_opt.alphas1_angle,
                decd_opt.alphas2_rho,
                decd_opt.alphas2_angle,
                decd_opt.phis,
                decd_opt.etas,
                decd_opt.thetas,)

#### Inspecting Gradients near minima

In [ ]:
@tf.function
def loss_fun(fids):
    #print('loss fun called')
    # I think it's important that the log is taken before the avg
    losses = tf.math.log(1 - fids)
    avg_loss = tf.reduce_sum(losses) / decd_opt.parameters["N_multistart"]
    return avg_loss

In [ ]:
variables = [decd_opt.betas_rho,
            decd_opt.betas_angle,
            decd_opt.gammas_rho,
            decd_opt.gammas_angle,
            decd_opt.phis,
            decd_opt.thetas]

with tf.GradientTape() as tape:
    fid = decd_opt.batch_fidelities(decd_opt.betas_rho,
            decd_opt.betas_angle,
            decd_opt.gammas_rho,
            decd_opt.gammas_angle,
            decd_opt.alphas1_rho,
            decd_opt.alphas1_angle,
            decd_opt.alphas2_rho,
            decd_opt.alphas2_angle,
            decd_opt.phis,
            decd_opt.etas,
            decd_opt.thetas,)
    new_loss = loss_fun(fid)
    #tape = tf.GradientTape()
    dloss_dvar = tape.gradient(new_loss, variables)
    print(dloss_dvar)
                   

In [ ]:
fid

In [7]:
@tf.function
def loss_fun(fids):
    #print('loss fun called')
    # I think it's important that the log is taken before the avg
    losses = tf.math.log(1 - fids)
    avg_loss = tf.reduce_sum(losses) / ecd_opt.parameters["N_multistart"]
    return avg_loss

In [8]:
variables = [ecd_opt.betas_rho,
            ecd_opt.betas_angle,
            ecd_opt.phis,
            ecd_opt.thetas]

with tf.GradientTape() as tape:
    fid = ecd_opt.batch_fidelities(ecd_opt.betas_rho,
            ecd_opt.betas_angle,
            ecd_opt.alphas_rho,
            ecd_opt.alphas_angle,
            ecd_opt.phis,
            ecd_opt.thetas,)
    new_loss = loss_fun(fid)
    #tape = tf.GradientTape()
    dloss_dvar = tape.gradient(new_loss, variables)
    print(dloss_dvar)
                   

[None, None, <tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[ 1.1329668e-03, -1.3888630e-02, -6.9238399e-03, -2.1846467e-03,
         5.4423166e-03,  4.3989962e-04, -1.4169593e-04,  1.2365915e-03,
         1.5760704e-04, -9.4283139e-03],
       [-4.5657158e-05,  4.1909516e-09,  0.0000000e+00, -1.2367964e-06,
         3.1139702e-05, -1.3265759e-05,  6.3851476e-06, -1.7061830e-06,
        -6.1746687e-07,  1.7229468e-08],
       [-1.7017126e-05, -2.9802322e-08,  6.5192580e-09, -5.1781535e-07,
        -4.3891370e-05,  3.5886467e-04,  2.9820949e-05,  2.4586916e-06,
         3.2596290e-08, -4.0978193e-08],
       [ 1.6234815e-05, -1.8626451e-09, -7.4505806e-09,  5.2154064e-07,
         7.6200813e-06, -8.2727522e-05, -1.2241304e-05, -4.8676215e-05,
        -2.0861626e-07, -2.7939677e-09],
       [ 8.7544322e-08,  1.1175871e-08, -9.3132257e-09,  1.1287630e-06,
        -3.0655414e-05,  2.2329018e-04,  1.9708648e-05,  2.7194619e-06,
         1.6763806e-08,  1.0058284e-07]], dtype=float3

In [ ]:
fid